In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
!nvidia-smi -L

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6b",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("patent/LexGPT-6B")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Load the datasets
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="../data/preprocessed/train_with_reasoning.csv",
    block_size=128,
)

val_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="../data/preprocessed/val_with_reasoning.csv",
    block_size=128,
)

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
from transformers import Trainer, TrainingArguments

# Define the parameters for fine-tuning
lr = 2e-5
end_lr = 4e-6
num_train_epochs = 1
warmup_steps = 100

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='lexgpt-with-reasoning-results',
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=warmup_steps,
        num_train_epochs=num_train_epochs,
        learning_rate=lr,
        weight_decay=0.1,
        fp16=True,
        logging_steps=10,
        num_train_epochs=1,
        logging_dir='./logs',
    ),
    data_collator=data_collator,
    train_dataset=train_dataset,
)
model.config.use_cache = False

In [ ]:
trainer.train()
trainer.save_model()

In [ ]:
model.push_to_hub("harsha28/lexgpt-lfqa-with-reasoning",
                  use_auth_token=True,
                  commit_message="lr 2e-5, 1 epoch",
                  private=True)